Nous allons réviser des notions de c++ et en apprendre de nouvelles à partie d'un exercice sur les graphes (parce que ca sert toujours d'avoir vu les graphes, ca permet de comprendre les algos récursifs qui utilisent la pile d'exécution et le passage à leur version itérative, mais ce sera pour plus tard).

Nous vous proposons à travers de notebook de réviser ou d'apprendre:
   - lecture simple dans un fichier
   - classes, constructeurs, destructeur
   - utilisation de tables de hash, vecteurs, string, stack, set
   - alias de type avec (using et pointeur sur fonction)
   - defaulted constructeur
   - deleted constructeur de copie
   - utilisation d'arguments de main
   - 

Supposons que nous voulons gérer des graphes. Un graphe est constitué d'un ensemble de sommets et de transitions entre ces sommets. Nous nous intéressons aux graphes orientés (i.e. dont les transitions ont un sommet de départ et un sommet d'arrivée), dont les sommets sont nommés et dont les transitions portent une étiquette, par exemple une valeur entière, cette classe sera une bonne candidate à être template sur le type des étiquetes mais ne faisons pas tout en même temps, nous allons procéder par petites étapes.

 je mets le graphe suivant dans le fichier petit_graphe.txt ou toute autre extension ... 
 
```c++
un deux 12
un trois 13
deux trois 23
trois quatre 34
quatre un 41
quatre deux 42
deux quatre 24
```

Voici un graphe: il y a quatre sommets de noms "un", "deux", "trois" et "quatre" et 7 transitions, la première va du sommet "un" au sommet "deux" et a pour valeur 12, la seconde va du sommet "un" au sommet "trois" et a pour valeur 13 etc.

Nous allons vous demander de proposer les classes c++ pour représenter un graphe dans la mémoire d'un ordinateur en vue d'opération comme le parcours du graphe ou du dessin du graphe. Le plus simple est d'avancer étapes par étapes.

Tout d'abord il nous faut des graphes pour tester notre code. Nous n'allons pas les écrire à la main mais bien sûr les lire dans des fichiers. Le format proposé dans l'exemple ci-dessus nous convient tout à fait, il est très lisible pour nous et pour l'ordinateur. La première chose que nous allons vous demander c'est faire une fonction qui ouvre un fichier et y lit le graphe, et pour l'instant affiche simplement ce qu'elle lit.

Réfléchissez et dans la cellule ci-dessous vous avez de l'aide.

Pour vous aider voici un exemple de lecture très simple dans un fichier qui contient des lignes composées d'une chaîne de caractères suivie d'un entier (on considère les lignes sont bien formées).

```c++
#include <iostream>
#include <string>
#include <fstream>

int main () {
  std::string file_name = "tutu.txt";
  std::ifstream file (file_name);
  if (not file) {
    std::cout << "something went wrong when trying to open the file"
	      << file_name
	      << std::endl; 
  } else {
    std::string name;
    int value;
    while (file >> name >> value) {
      std::cout
	<< "we red " << name << " and " << value << std::endl;
    }
  }
  return 0;
}
```

C'est un peu bête d'avoir mis le nom du fichier en *dur*  `tutu.txt` peut être serait-il bien de prendre comme nom du fichier le premier argument de la ligne de commande en vérifiant bien sûr qu'il existe.

Essayez et ci-dessous vous avez de l'aide.

```c++
#include <iostream>

int main (int argc, char* argv []) {
  if (argc > 1)
    std::cout << argv[1] << std::endl;
  return 0;
}
```

Maintenant que nous savons ouvrir un fichier et séparer chacune des ses lignes en deux noms de sommets et la valeur de la transition du premier au second, le constructeur des graphes pourra prendre en argument le nom d'un fichier et construire son graphe à partir du contenu du fichier (nous considérons que le fichier est bien formé, mais rien ne vous empêche de réfléchir aux potentielles erreurs dès maintenant).

Il est temps de commencer nos classes. Nous avons besoin d'une classe `Graph` et d'une classe `Vertex` et celle-ci a besoin d'une classe `Edge`. Que mettre dans ces classes ?

Nous pouvons dire que chaque sommet connaît les transitions dont il est le sommet de départ, on appelle cette représentation une représentation des graphes en *listes d'adjacence* (en c++, on va préférer utiliser des vecteurs, plutôt que des listes, ce sont de bien meilleures représentations). Notons qu'un autre manière de représenter un graphe serait sa *matrice d'adjacence* (où $G[i, j]$ contient la valeur de la transition entre le sommet $i$ et le sommet $j$, si elle existe). Les sommets portent un nom et savent ajouter une transition à leur liste de transitions, cela paraît bien.

Réfléchissez à votre classe `Vertex`, la cellule ci-dessous vous donnera un exemple simple d'un struture pour la solution. Je fais en sorte que le code compile (avec `-c` pour compile-only bien sûr) mais bien sûr il va lui manquer pas mal de choses comme la définition des fonctions que je vous engage à faire, sur ce code ou sur le vôtre.

```c++
#include <iostream>
#include <vector>
class Edge; // nous disons que la classe Edge existe sans dire encore comment elle est faite
class Vertex {
  friend  std::ostream& operator<< (std::ostream&, const Vertex&); // on pense à l'affichage des sommets 
  friend class Edge; // on va permettre à la classe Vertex d'accéder aux champs privés de la classe Edge 
                     // puisque ces deux classes sont intimement liées
private:
  std::string name; // le nom des sommets
  std::vector<Edge> adjacency_list; // et la liste de ses transitions dont ce sommet est le sommet de départ
public:
  Vertex (std::string name); // un constructeur très simple avec une liste de transitions vide
  void add (Edge edge); // la fonction ppur ajouter les transitions dans la liste d adjacence du sommet
};
```

Le graphe va contenir ses sommets et ses transitions et nous savons que le constructeur des graphes va prendre en argument un nom de fichier et construire le graphe contenu dans le fichier.

ainsi le constructeur de graphe, ouvre le fichier, y lit les transitions les une après les autres. Quand il lit un nom de sommet, il y a deux cas: ce sommet n'existe pas encore, le graphe doit le créer et le rajouter à ses sommets; ou le sommet existe déjà, le graphe doit retrouver l'objet dans lequel il a rangé ce sommet. Trouver un objet à partir d'un identifiant qui désigne cet objet de manière unique, ca ne vous rappelle rien ? les fameux couples $(clé, valeur)$ ? les ... dictionnaires oui ! Naturellement les dictionnaires sont les meilleurs conteneurs dès qu'il s'agit de rechercher des éléments par leur clé ! comme ici rechercher le sommet par son nom. La recherche d'un sommet sera ainsi beaucoup plus rapide que si les sommets avaient été mis dans une liste ! elle se fera en temps ??? en temps ??? en temps constant ! (si la fonction de hashage est bien adaptée à votre problème bien sûr) alors que dans une liste chaînée la recherche d'un élément se fera en temps ??? en temps ??? en temps linéaire bien sûr.

Le dictionnaire encore appelé *table de hash* ou *map* ou (pour c++ qui se distingue toujours) *unordered_map* http://www.cplusplus.com/reference/unordered_map/unordered_map/ c'est la première structure de données à laquelle il faut penser ! les fameux `dict` et `set` en python... vous allez utiliser une table de hash pour stocker vos sommets dans votre graphe.

Dans la cellule ci-dessous vous avez de l'aide sous la forme d'un exemple d'utilisation d'une table de hash en c++.

```c++
#include <iostream>
#include <unordered_map>

int main (int argc, char* argv []) {
  // type de la clé est std::string et type de la valeur est int
  // à l'initialisation de la table de has je lui donne des couples (clé, valeur)
  std::unordered_map<std::string, int> um = {{"six", 6}, {"sept", 7}};

  // mais je peux aussi rajouter des éléments après
  um["un"] = 1;
  um["deux"] = 2;
  um["trois"] = 3;
  um["quatre"] = 4;
  um["cinq"] = 5;

  // les couples (clé, valeur) sont en c++ des paires std::pair où la clé est le
  // champ first et la valeur le champ second.

  // Je vais parcourir tous les éléments de la table de hash um,
  // j'utilise le for-auto de c++ qui est drôlement pratique
  // (attention c'est uniquement à partir de c++11) !
  
  for (auto e: um) {
    std::cout << (e).first // the key value (of type Key)
	      << " "
	      << (e).second // the key value (of type Key)
	      << std::endl;
  }

  // Alors vous voyez que l'ordre dans lequel les paires (clé,
  // valeur) sont listées de la table de hash ne respecte pas l'ordre d'insertion
  // (qui n'a pas de sens c'est la fonction de hash qui a décidé de
  // l'ordre des éléments).

  // Et si je n'utilise pas le for-auto ? Et bien c'est un peu plus
  // pénible à écrire mais ca se fait aussi. J'initialise un itérateur
  // sur le début de la table de hash grâce à begin et je vais parcourir
  // les éléments jusqu'après le dernier élément qui est end.
  // Lorsque la table de hash sur laquelle on veut itérer est
  // vide, c'est que begin et égal à end.
  
  std::unordered_map<std::string, int>::iterator it;
  for (it = um.begin(); it != std::end(um); ++it) {
    std::cout << it->first // the key value (of type Key)
	      << " "
	      << it->second // the key value (of type Key)
	      << std::endl;
  }

  // Pour que le for soit presque comme les for classiques, ils ont
  // surchargés l'opérateur ++ des itérateurs sur conteneurs.

  // Les conteneurs en c++ possèdent donc des mécanismes d'iterateurs
  // sur leurs objets, ce qui permet aussi de rechercher un élément
  // dans une table de hash.

  // "un" existe dans la table, je le cherche et j'affiche sa valeur
  std::unordered_map<std::string, int>::iterator e_oui = um.find("un");
  std::cout << e_oui->second << std::endl;
  
  // "dix" n'existe pas dans la table, on le recherche
  std::unordered_map<std::string, int>::iterator e_non = um.find("dix");
  // donc e_non->first et e_non->second n'existent pas, aussi en cherchant "dix"
  // l'itérateur pour dire qu'il ne l'a pas trouvé nous retourne end.

  if (e_non == um.end()) {
    std::cout << "dix n'est pas une clé de la table de hash\n";
  }

  return 0;
}


```

Voila vous savez utiliser une table de hash simple en c++.

Alors là j'entends les paresseux qui en ont marre de taper le super looOOOoooonnnnngggg terme`std::unordered_map<std::string, int>` et qui aimeraient bien un raccourci... oui c++ permet de faire des alias pour les noms de types c'est à dire de prendre des synonymes (ce ne sont pas des types distincts, ce sont de nouveaux noms pour un type existant). Voici des exemples:

```c++
#include <unordered_map>

// le petit nom pour le type de nos tables de hash qui ont pour clé un std::string et pour valeur un int
using UMap = std::unordered_map<std::string, int>;

// une fonction avec un nom super loooonnnggg
void super_long_nom_pour_une_fonction (double) {}

int main () {

   UMap um = {{"six", 6}, {"sept", 7}}; // ah c'est beaucoup mieux !
    
   // et on peut donner d'autres exemples:
    
   using STR = const char *; // voici le type des chaînes de caractères c-style
   STR c = "Hello World ! ";
   
   using PFVD = void (*) (double); // voici le type pointeur vers une fonction
                                   // qui prend un double et ne rend rien
   PFVD bob = super_long_nom_pour_une_fonction;
   bob(13.56); // ah c'est plus court !

   // et ainsi de suite:    
   using PTABI10 = int [10]; // le type tableau de 10 entiers
   PTABI10 tab = {1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10};
    
   using RTABI10 = PTABI10 &;  // le type reference vers un tableau de 10 ints
   RTABI10 rtab = tab ;

   return 0;
}

```

Donc maintenant je veux stocker mes sommets dans une table de hash. Pour la clé de la table de hash ? facile c'est une chaîne de caractères c++ i.e. une `std::string`.  Pour la valeur ? il semble qu'un sommet soit indiqué mais quel type exactement ? directement un objet ou l'adresse d'un objet ?

Et bien on essaie de faire une table de hash pour stocker des objets de type `A`. On a le choix entre stocker des `A` ou des adresses d'objets de type `A`.

Voici `A` 

```c++
// in file A.h
#ifndef A_H
#define A_H

#include <iostream>

class A {
  friend std::ostream& operator<< (std::ostream& os, const A& ra);

  int valeur;
public:
  A (int valeur = 0) : valeur(valeur) {}
};

inline std::ostream& operator<< (std::ostream& os, const A& ra) {
  os << "A(" << ra.value << ")";
  return os;
}
#endif
```

Donc j'essaie la table de hash sur des objets de type `A`

```c++
#include <iostream>
#include <unordered_map>

#include "A.h" // on importe A

int main (int argc, char* argv []) {
  // je crée un objet de type A
  A un_a(81);

  // je crée une table de hash d'objets de type A et j'y mets l'objet un_a dedans avec comme clé "coucou"
  std::unordered_map<std::string, A> uA = {{"coucou", un_a}};

  // j'incrémente la valeur de l'objet de la table de hash uA dont "coucou" est la clé
  uA["coucou"].incr();

  // je l'affiche
  std::cout << uA["coucou"] << std::endl; // il vaut bien "A(82)"jusque là tout va bien !

  // j'affiche le un_a que j'ai mis dans la table de hash
  std::cout << un_a << std::endl;
  
  // ah ben ... un_a vaut toujours "A(81)" et pas "A(82)" !
  
  // ce n'est pas ce que je voulais !
    
  // mais c'est ce que j'ai demandé: de travailler sur des objets
  // donc quand je mets un objet de type A ici un_a dans la table de
  // hash, il est copié, dans la table de hash j'ai une copie de un_a
  // et c'est la copie qui a été modifiée pas un_a !
    
  return 0;
}
```



Que m'est-t-il arrivé ? Comme j'ai demandé à stocker des objets de type `A` dans la table de hash, quand je mets un objet (comme `un_a`) il est recopié. Dans la table de hash je me retrouve avec une copie de mon objet (ici une copie de `un_a` mais pas l'objet `un_a` lui même), je ne peux donc pas modifier l'objet en passant par la table de hash mais une simple copie.

Bon vous avez compris, il va falloir d'une manière ou d'une autre l'adresse de l'objet si vous voulez pouvoir le modifier à travers la table de hash.

Nous avancons: le constructeur des graphes lit le fichier et construit les sommets les uns après les autres en stockant dans une table de hash, les paires *(non du sommet, adresse du sommet)*. Cette table de hash est une donnée de la classe `Graph`.

Alors attention, si dans le constructeur des graphes, vous remplissez votre table de hash avec des adresses de sommets, les objets doivent continuer à exister en dehors de la fonction qui les crée ! Vous n'avez pas compris ? Essayez de voir le problème dans le code suivant: 

```c++
// in file main.cpp 
#include <iostream>
#include <unordered_map>

#include "A.h"

void je_remplis_ma_table_de_hash (std::unordered_map<std::string, A*>& rmap) {
  // attention les yeux ce code est archi faux !!!
  A a (1);
  rmap["un"] = &a; // ca fait vraiment super mal aux yeux !
}

int main (int argc, char* argv []) {
  A b(2);
  std::unordered_map<std::string, A*> uA = {{"deux", &b}};
  je_remplis_ma_table_de_hash(uA);

  // et j'affiche tout le monde ... qu'est ce qui ne va pas ?
  // qu'est ce qui est à l'adresse e.second ?
  for (auto e: uA) {
    std::cout << *(e.second) << std::endl;
  }
  
  return 0;
}
```

```c++
$g++ main.cpp -o boum
$ ./boum
ici rien ne doit se passer comme attendu
```

```c++
// pour les curieux ce que valgind nous dit (dans son jargon)
$ ./valgrind ./boum
==27175== Memcheck, a memory error detector
==27175== Copyright (C) 2002-2017, and GNU GPL'd, by Julian Seward et al.
==27175== Using Valgrind-3.13.0 and LibVEX; rerun with -h for copyright info
==27175== Command: ./boum
==27175== 
==27175== Conditional jump or move depends on uninitialised value(s)
==27175==    at 0x4F43B2A: std::ostreambuf_iterator<char, std::char_traits<char> > std::num_put<char, std::ostreambuf_iterator<char, std::char_traits<char> > >::_M_insert_int<long>(std::ostreambuf_iterator<char, std::char_traits<char> >, std::ios_base&, char, long) const (in /usr/lib/x86_64-linux-gnu/libstdc++.so.6.0.25)
==27175==    by 0x4F50074: std::ostream& std::ostream::_M_insert<long>(long) (in /usr/lib/x86_64-linux-gnu/libstdc++.so.6.0.25)
==27175==    by 0x1092CF: operator<<(std::ostream&, A const&) (in /home/JohnDoe/cplusplus/boum)
==27175==    by 0x10954E: main (in /home/JohnDoe/cplusplus/boum)
==27175== 
==27175== Use of uninitialised value of size 8
==27175==    at 0x4F4362E: ??? (in /usr/lib/x86_64-linux-gnu/libstdc++.so.6.0.25)
==27175==    by 0x4F43B53: std::ostreambuf_iterator<char, std::char_traits<char> > std::num_put<char, std::ostreambuf_iterator<char, std::char_traits<char> > >::_M_insert_int<long>(std::ostreambuf_iterator<char, std::char_traits<char> >, std::ios_base&, char, long) const (in /usr/lib/x86_64-linux-gnu/libstdc++.so.6.0.25)
==27175==    by 0x4F50074: std::ostream& std::ostream::_M_insert<long>(long) (in /usr/lib/x86_64-linux-gnu/libstdc++.so.6.0.25)
==27175==    by 0x1092CF: operator<<(std::ostream&, A const&) (in /home/JohnDoe/cplusplus/boum)
==27175==    by 0x10954E: main (in /home/JohnDoe/cplusplus/boum)
==27175== 
==27175== Conditional jump or move depends on uninitialised value(s)
==27175==    at 0x4F4363B: ??? (in /usr/lib/x86_64-linux-gnu/libstdc++.so.6.0.25)
==27175==    by 0x4F43B53: std::ostreambuf_iterator<char, std::char_traits<char> > std::num_put<char, std::ostreambuf_iterator<char, std::char_traits<char> > >::_M_insert_int<long>(std::ostreambuf_iterator<char, std::char_traits<char> >, std::ios_base&, char, long) const (in /usr/lib/x86_64-linux-gnu/libstdc++.so.6.0.25)
==27175==    by 0x4F50074: std::ostream& std::ostream::_M_insert<long>(long) (in /usr/lib/x86_64-linux-gnu/libstdc++.so.6.0.25)
==27175==    by 0x1092CF: operator<<(std::ostream&, A const&) (in /home/JohnDoe/cplusplus/boum)
==27175==    by 0x10954E: main (in /home/JohnDoe/cplusplus/boum)
==27175== 
==27175== Conditional jump or move depends on uninitialised value(s)
==27175==    at 0x4F43B86: std::ostreambuf_iterator<char, std::char_traits<char> > std::num_put<char, std::ostreambuf_iterator<char, std::char_traits<char> > >::_M_insert_int<long>(std::ostreambuf_iterator<char, std::char_traits<char> >, std::ios_base&, char, long) const (in /usr/lib/x86_64-linux-gnu/libstdc++.so.6.0.25)
==27175==    by 0x4F50074: std::ostream& std::ostream::_M_insert<long>(long) (in /usr/lib/x86_64-linux-gnu/libstdc++.so.6.0.25)
==27175==    by 0x1092CF: operator<<(std::ostream&, A const&) (in /home/JohnDoe/cplusplus/boum)
==27175==    by 0x10954E: main (in /home/JohnDoe/cplusplus/boum)
==27175== 
A(0)
A(2)
==27175== 
==27175== HEAP SUMMARY:
==27175==     in use at exit: 0 bytes in 0 blocks
==27175==   total heap usage: 6 allocs, 6 frees, 73,896 bytes allocated
==27175== 
==27175== All heap blocks were freed -- no leaks are possible
==27175== 
==27175== For counts of detected and suppressed errors, rerun with: -v
==27175== Use --track-origins=yes to see where uninitialised values come from
==27175== ERROR SUMMARY: 4 errors from 4 contexts (suppressed: 0 from 0)
$
```

on peut ne pas très bien le comprendre mais on comprend que clairement quelque chose ne lui plaît pas, il est très difficile de faire du c++ sans un outil comme `valgrind` voire impossible

oui clairement nous stockons dans la table de hash, l'adresse d'un objet `a` de type `A` qui est construit en local dans la fonction `je_remplis_ma_table_de_hash`. Il a donc été créé dans la pile d'execution, il a ensuite été dépilé à la sortie de la fonction, et votre adresse stockée ne pointe plus un objet existant d'où le très clair `Use of uninitialised value of size 8` quand vous essayez d'afficher `a`. A cet endroit y'a plus d'objet ! Vous vous rappelez de `new` ? il va falloir l'utiliser et attention à bien faire `delete` à la fin de la vie du graphe.

Alors où en sommes nous ? Vous avez une classe `Graph` qui contient un objet de type table de hash dans laquelle vous stockez les adresses des sommets que vous allouez en mémoire dynamique (le tas ou encore *heap*). Dans ces sommets vous avez mis une liste de transitions.

Réfléchissez à votre classe `Graph`, la cellule ci-dessous vous donnera un exemple simple de solution.

```c++
#include <iostream>
#include <string>
#include <unordered_map>

class Vertex;

using UMap = std::unordered_map<std::string, Vertex*>;

class Graph {
  friend std::ostream& operator<< (std::ostream&, const Graph&);
  UMap vertices;

private:
  
  // cette fonction recherche si le sommet de nom name existe déjà dans la table de
  // hash si oui elle renvoie son adresse, si non elle le crée, le met dans la lable
  // de hash et renvoie son adresse
  Vertex* find_vertex (std::string& name);
	  
public:

  // cette fonction lit le graphe sans le fichier file_name et construit les sommets et ses transitions  
  Graph (const char* file_name);
  
  // cette fonction parcourt les sommets de la table de hash et les
  // détruit puisque quelque part un new a été fait...
  ~Graph ();
};
```

Parlons destructeur: comme le constructeur de `Graph` alloue en mémoire dynamique des objets de type `Vertex`, le desctructeur de `Graph` doit parcourir les sommets de la table de hash pour les détruire.

Que sont les transitions ? Les objets de type `Edge` et bien une transition connaît son sommet de départ (ben non en fait ca ne sert à rien ! puisqu'elle sera ajoutée dans le vecteur des transitions de son sommet de départ, ce serait redondant...) et son sommet d'arrivé et elle est rangée dans la liste des transitions de son sommet de départ. Dans les `Edge` attention à ne pas recopier les sommets... mais bien à prendre leur adresse.

```c++
#include <iostream>

class Vertex;
class Edge {
  friend  std::ostream& operator<< (std::ostream&, const Edge&);
  friend class Vertex;
private:
  Vertex* end;
  int value;
public:
  Edge (Vertex* end, int value):
    end(end), value(value) {
  }
};
```

Et la classe des transitions pourrait être template sur le type des valeurs des transitions, les plus motivés pourront essayer de le faire dès que leur code sur `int` fonctionne

Et enfin quand le constructeur de `Graph` est terminé, vous avez dans votre table de hash tous les sommets de votre graphe. Je vous propose, pour tester de redéfinir l'`operator<<` sur les `Graph` (en la mettant `friend` de la classe `Graph` ou autre manière mais ne permettez pas au `public` d'accéder à votre table de hash ! elle doit être privée à votre classe `Graph`).

Pour simplifier, mettez toutes les définitions de vos classes dans un même fichier `graph.h`, toutes les fonctions ne pourront pas être definies à l'intérieur des `{}` des classes (vous avez des inter-dépendances qui vont l'empêcher), mettez les `inline` en dehors de leur classe au bons endoits (quand toutes les classes nécessaires ont été définies).

Mettez tout en privé sauf les constructeurs, les destructeurs et les fonctions qui sont demandées comme l'affichage ou les parcours.

Ensuite n'implémentez pas de constructeurs de copie ni d'opérateur d'affectation, déclarez les `delete` afin de les interdire comme dans l'exemple suivant:  

```c++
// in file nocopy.cpp
class NoCopy {
public:
  NoCopy (const NoCopy&) = delete; // j'interdis la copie d'objets de type NoCopy
                                   // interdire veut dire: empêcher le code de compiler
    
  NoCopy () = default; // je demande à avoir la version générée par c++ pour le constructeur sans argument
                       // pourquoi ai-je besoin de cela ?
                       // parce que comme j'ai interdit la copie j'ai parlé du constructeur de copie
                       // donc c++ arrête de me donner implicitement le constructeur sans argument
                       // et comme je le veux et que je veux la version de c++ qui est mieux que la mienne
 // NoCopy () {} non je ne le définis pas, je préfère laisser c++ travailler... (voir ci-dessus)
};

void foo (NoCopy) {} // je veux interdire le passage d'arguments par copie d'objets de type NoCopy

NoCopy bar () {      // je veux interdire le retour de fonction par copie d'objets de type NoCopy
  return NoCopy();
}

int main () {
  NoCopy nc;
  NoCopy nc1 = nc; // interdit !! (le programme ne va pas compiler)
  foo(nc); // interdit !! (le programme ne va pas compiler)
  NoCopy nc2 = bar(); // interdit !! (le programme ne va pas compiler)
  return 0;
}
```

clairement j'ai réussi à interdire la copie sous toute ses formes...

```c++
$ g++ npcopy.cpp
nocopy.cpp: In function ‘NoCopy bar()’:
nocopy.cpp:10:17: error: use of deleted function ‘NoCopy::NoCopy(const NoCopy&)’
   return NoCopy();
                 ^
nocopy.cpp:4:3: note: declared here
   NoCopy (const NoCopy&) = delete;
   ^~~~~~
nocopy.cpp: In function ‘int main()’:
nocopy.cpp:15:16: error: use of deleted function ‘NoCopy::NoCopy(const NoCopy&)’
   NoCopy nc1 = nc;
                ^~
nocopy.cpp:4:3: note: declared here
   NoCopy (const NoCopy&) = delete;
   ^~~~~~
nocopy.cpp:16:9: error: use of deleted function ‘NoCopy::NoCopy(const NoCopy&)’
   foo(nc);
         ^
nocopy.cpp:4:3: note: declared here
   NoCopy (const NoCopy&) = delete;
   ^~~~~~
nocopy.cpp:7:6: note:   initializing argument 1 of ‘void foo(NoCopy)’
 void foo (NoCopy) {}
      ^~~
nocopy.cpp:17:20: error: use of deleted function ‘NoCopy::NoCopy(const NoCopy&)’
   NoCopy nc2 = bar()
                    ^
nocopy.cpp:4:3: note: declared here
   NoCopy (const NoCopy&) = delete;
   ^~~~~~
   
Compilation exited abnormally with code 1 at Sun May 10 18:57:44
```

Voici un exemple de main ainsi que ce qu'il devrait afficher (par exemple):

```c++
#include "graph.h"
int main (int argc, char* argv[]) {
  if (argc >= 1) {
    Graph g(argv[1]);
    std::cout << g << std::endl;
  }
  return 0;
}
```
```c++
$ g++ main.cpp -o graph
$ ./graph petit_graphe.txt

quatre -41-> un
quatre -42-> deux
trois -34-> quatre
un -12-> deux
un -13-> trois
deux -23-> trois
deux -24-> quatre

$
```

Voici au cas où dans la cellule ci-dessous une petite correction du graphe sans les parcours et du main.

in file graphe.h

```c++
#ifndef GRAPHE_H
#define GRAPHE_H

#include <iostream>
#include <string>
#include <fstream>
#include <stdexcept>
#include <unordered_map>
#include <vector>

class Vertex;
class Edge {
  friend  std::ostream& operator<< (std::ostream&, const Edge&);
  friend class Vertex;
private:
  Vertex* end;
  int value;
public:
  Edge (Vertex* end, int value):
    end(end), value(value) {
  }
};

class Vertex {
  friend std::ostream& operator<< (std::ostream&, const Vertex&);
  // pour que l'affichage de Edge puisse accéder à end->name
  friend std::ostream& operator<< (std::ostream& os, const Edge&);
  friend class Edge;
private:
  std::string name;
  std::vector<Edge> adjacency_list;
public:
  Vertex (std::string name): name(name) {
  }
  void add (Edge edge) {
    adjacency_list.push_back(edge);
  }
};

using UMap = std::unordered_map<std::string, Vertex*>;

class Graph {
  friend std::ostream& operator<< (std::ostream&, const Graph&);
  UMap vertices;

private:
  Vertex* find_vertex (std::string& name);
	  
public:
  Graph (const char*);
  ~Graph ();

};

inline Vertex* Graph::find_vertex (std::string& name) {
  UMap::iterator vertex_it = vertices.find(name);
  // vertex_it est différent de end quand name est la clé d'un élément
  // de la table de hash donc qu'un sommet portant ce nom existe déjà.
  if (vertex_it != vertices.end())
    return vertex_it->second;
  // name n'est pas la clé d'un élément de la table de hash, le sommet
  // de ce nom n'existe pas, je dois le créer
  Vertex* vertex = new Vertex(name);
  // je le range dans la table de hash
  vertices[name] = vertex;
}

inline Graph::Graph (const char* file_name) {
  if (file_name) {
    // si le nom du fichier existe, j'ouvre le fichier en lecture
    std::ifstream file (file_name);
    if (not file) {
      // l'ouverture s'est mal passée soit le fichier n'existe pas, soit
      // nous n'avons pas des droits de lecture, soit...
      std::string msg = "something went wrong when trying to open the file";
      // je lance une exception
      throw std::runtime_error(msg+file_name);
    } else {
      // le fichier existe, je considère qu'il est bien formé,
      // j'attends donc sur chaque ligne une transition avec le nom du
      // sommet de départ, le nom du sommet d'arrivée et la valeur de
      // la transition
      std::string vertex_begin, vertex_end;
      int value;
      // dans le flux d'entrée correspondant au fichier (file de type
      // ifstream) je lis ces trois objets
      while (file >> vertex_begin >> vertex_end >> value) {
	// je cherche le sommet de départ
	Vertex* vertex1 = find_vertex(vertex_begin);
	// je cherche le sommet d'arrivée
	Vertex* vertex2 = find_vertex(vertex_end);
	// je crée la transition
	vertex1->add(Edge(vertex2, value));
      }
    }
  }
}

inline Graph::~Graph () {
  for (auto e: vertices) {
    delete e.second;
  }
}

inline std::ostream& operator<< (std::ostream& os, const Graph& rg) {
  for (auto e: rg.vertices)
    os << *e.second;
  return os;
}

inline std::ostream& operator<< (std::ostream& os, const Vertex& rv) {
  for (auto& e: rv.adjacency_list)
    os << rv.name << e << std::endl;
  return os;
}

inline std::ostream& operator<< (std::ostream& os, const Edge& re) {
  os << " -" << re.value << "-> " << re.end->name;
  return os;
}
#endif
```

------------------

in file main.cpp

```c++
#include <iostream>
#include "graphe.h"

int main (int argc, char* argv[]) {
  if (argc >= 1) {
    Graph g(argv[1]);
    std::cout << g << std::endl;
  }
  return 0;
}
```

------------------
in file petit_graphe.txt
```c++
un deux 12
un trois 13
deux trois 23
trois quatre 34
quatre un 41
quatre deux 42
deux quatre 24
```

------------------
on compile et on run

```c++
$ g++ main.cpp -o graphe
$ ./graphe petit_graphe.txt
quatre -41-> un
quatre -42-> deux
trois -34-> quatre
un -12-> deux
un -13-> trois
deux -23-> trois
deux -24-> quatre
```


Passons maintenant à l'implémentation d'algorithmes de parcours de graphes. Le premier est le plus célèbre est le parcours du graphe en profondeur, je suis sûre que certains d'entre vous le connaissent déjà. 

L'idée du parcours d'un graphe en profondeur est plutôt simple. Vous allez partir de tous les sommets du graphe (au cas où le graphe possède des sommets non connectés aux autres il faut bien les atteindre tous) et à partir de chaque sommet vous allez parcourir tous ses descendants.

Donc vous prenez le premier sommet du graphe (appelons le "lulu le sommet") et vous appelez sur ce sommet la fonction `dfs`. La fonction commence par vérifier que c'est bien la première fois que vous voyez ce sommet, bon vous venez tout juste de commencer le parcours ca devrait être le cas (sinon vous buggez). La fonction marque alors le sommet "lulu" comme déjà-vu et bien oui vous ne voulez pas tourner en rond indéfiniment si vous le rencontriez de nouveau pendant le parcours de ses descendants (ce qui se passerait si vous essayez de parcourir le morceaus de graphe lulu->a->b->c->lulu sans marquer les sommets).

Ensuite vous regardez les transitions partant de "lulu". Si il n'y en a pas (i.e. le sommet n'a pas de sommets adjacents c'est un cul-de-sac), votre parcours est terminé c'était super rapide. Si des transitions partent de "lulu", vous prenez la première transition, et vous appelez sur son sommet d'arrivée la fonction `dfs` ... tout simplement ... et oui cette fonction profite de la récursivité. Bref au bout d'un certain temps tous les descendants du sommet d'arrivée de la première transition auront été vus (sinon c'est que vous buggez), vous prenez alors la seconde transition partant de "lulu" et vous lancez `dfs` sur son sommet d'arrivée ... quand toutes les transitions, dont "lulu" est le sommet de départ, auront été parcourues, vous avez terminé le parcours du graphe à partir du sommet "lulu". Et vous regardez un autre sommet. Au cas où mes explications vous semblent confuses, lisez là https://en.wikipedia.org/wiki/Depth-first_search.

Alors une question: *Comment allez-vous marquer les sommets comme déjà-vus ?* C'est là qu'intervient le type ensemble `set` de c++. Au moment où vous commencez le parcours du graphe, vous créez un ensemble que vous passez en argument à la fonction `dfs` (par référence, évitez de recopier l'ensemble ca marcherait beaucoup moins bien). À chaque fois que vous voulez *marquer un sommet* vous le mettez dans l'ensemble et les ensembles en c++ sont ... des tables de hash mais qu'avec des clés (facile).

Voici un petit exemple d'ensemble en c++.

```c++
// in file main.cpp
#include <iostream>
#include <set>

int main (int argc, char* argv []) {
  // l'ensemble de std::string comporte trois chaînes de caractères à sa construction
  std::set<std::string> str_set ={"un", "deux", "trois"}; // entre les {} c'est une liste d'initialisation
                                                          // nous ne parlons dans un prochain notebook
                                                          
  str_set.insert("quatre");
  // je cherche si "quatre" est dans l'ensemble
  std::set<std::string>::iterator el = str_set.find("un");
  
  // j'ai obtenu un iterator, je regarde si a bien un objet ou si il est vide (end)
  if (el != str_set.end())
    // l'objet est dans l'ensemble je l'enlève
    str_set.erase(el);
  
  // je parcours tous les objets de l'ensemble
  for (auto e: str_set) {
    std::cout << e << std::endl;
  }

  return 0;
}
```

```c++
$ g++ main.cpp -o set
$ ./set
deux
quatre
trois
```

Maintenant vous implémentez des fonctions `dfs`, la première dans la classe `Graph` elle va commencer par créer un objet de type `std::set` vide puis parcourir tous les sommets de la table de hash et appeler `dfs` sur chacun en lui passant une référence sur l'ensemble, et l'autre `dfs` sera dans la classe `Vertex` elle va prendre en argument la référence sur l'ensemble et faire tout ce dont nous avons déjà parlé.

Pour debugger, quand vous appelez `dfs` la première fois sur un sommet affichez son nom et suivez par où le parcours passe.

Voici dans la cellule ci-dessous une petite correction pour les fonctions `dfs`.

je vous donne juste les changements

dans le fichier `graphe.h` changement pour la classe `Vertex`

```c++

class Vertex {
  friend std::ostream& operator<< (std::ostream&, const Vertex&);
  friend std::ostream& operator<< (std::ostream& os, const Edge&);
  friend class Edge;
private:
  std::string name;
  std::vector<Edge> adjacency_list;
public:
  Vertex (std::string name): name(name) {
  }
  void add (Edge edge) {
    adjacency_list.push_back(edge);
  }
  void dfs (std::set<std::string>&);   // LÀ
};

inline void Vertex::dfs (std::set<std::string>& mark) { //  ET LÀ
  if (mark.find(name) == mark.end()) {
    mark.insert(name);
    for (auto& t: adjacency_list) {
      t.end->dfs(mark);
    }
  }
}
```

-----------------------------------------

in file main.cpp
dans le fichier `graphe.h` changement pour la classe `Graph`

```c++
class Graph {
  friend std::ostream& operator<< (std::ostream&, const Graph&);
  UMap vertices;

private:
  Vertex* find_vertex (std::string& name);
	  
public:
  Graph (const char*);
  ~Graph ();
  void dfs ();  // LÀ
};

inline void Graph::dfs() { // ET LÀ
  std::set<std::string> mark;
  for (auto& s: vertices) {
    s.second->dfs(mark);
  }
}
```
-----------------------------------------

in file main.cpp

```c++
#include <iostream>
#include "graphe.h"

int main (int argc, char* argv[]) {
  if (argc >= 1) {
    Graph g(argv[1]);
    g.dfs();  // LÀ
  }
  return 0;
}
```

-----------------------------------------
in file petit_graphe.txt
```c++
un deux 12
un trois 13
deux trois 23
trois quatre 34
quatre un 41
quatre deux 42
deux quatre 24
```

-----------------------------------------
la compilation et le run

```c++
$ g++ main.cpp -o graphe
$ ./graphe petit_graphe.txt
quatre, un, deux, trois,
```

Ce serait bien de rajouter une fonction qui permette dans un graphe, de lancer un parcours `dfs` à partir d'un nom de sommet. Et bien pour cela, il suffit de rajouter dans la classe `Graph` une fonction `dfs` qui prend en argument une `std::string` qui regarde si un sommet de ce nom existe dans le graphe, si oui il lance le `dfs` sur le sommet et si non il lance une exception (non mais quand même l'utilisateur doit être prévenu de son erreur, soyons impitoyable envers lui !).

Et pour les `Vertex` ? il y aura une seconde fonction `dfs` qui ne prend pas d'argument mais qui crée un `std::set` et lance `dfs` sur les sommets d'arrivée de ses transitions en leur passant l'ensemble en argument (comme d'habitude).

Voici dans la cellule ci-dessous une petite correction pour les fonctions `dfs` à partir du nom d'un sommet

je vous donne juste les changements et les `dfs` à ajouter

dans le fichier `graphe.h` changement pour la classe `Vertex`

```c++
class Vertex {
  friend std::ostream& operator<< (std::ostream&, const Vertex&);
  friend std::ostream& operator<< (std::ostream& os, const Edge&);
  friend class Edge;
private:
  std::string name;
  std::vector<Edge> adjacency_list;
public:
  Vertex (std::string name): name(name) {
  }
  void add (Edge edge) {
    adjacency_list.push_back(edge);
  }
  void dfs ();  // LÀ
  void dfs (std::set<std::string>&);
};

inline void Vertex::dfs () { // ET LÀ
  std::set<std::string> mark = {name};
  std::cout << name << ", ";
  for (auto& s: adjacency_list) {
    s.end->dfs(mark);
  }
}
```
---------------------------

dans le fichier `graphe.h` changement pour la classe `Graph`

```c++

class Graph {
  friend std::ostream& operator<< (std::ostream&, const Graph&);
  UMap vertices;

private:
  Vertex* find_vertex (std::string& name);
	  
public:
  Graph (const char*);
  ~Graph ();
  void dfs ();
  void dfs (std::string);  // LÀ
};

// cette fonction lance le parcours du graphe à partir du sommet dont
// le nom est passé en argument, on ne parcoura que les descendants de
// ce sommet
inline void Graph::dfs (std::string name) { // ET LÀ
  UMap::iterator vertex_it = vertices.find(name);
  if (vertex_it != vertices.end()) {
    vertex_it->second->dfs();
  } else {
    // si le sommet n'est pas dans le graphe on lance une run time exception
    std::string msg = " vertex not in graph";
    throw std::runtime_error(name+msg);
  }
}
```
--------------------------------

in file main.cpp

```c++
#include <iostream>
#include "graphe.h"

int main (int argc, char* argv[]) {
  if (argc >= 1) {
    Graph g(argv[1]);
    g.dfs("un");  // LÀ
  }
  return 0;
}
```
--------------------------------

in file petit_graphe.txt
```c++
un deux 12
un trois 13
deux trois 23
trois quatre 34
quatre un 41
quatre deux 42
deux quatre 24
```
--------------------------------

la compilation et le run

```c++
$ g++ main.cpp -o graphe
$ ./graphe petit_graphe.txt
un, deux, trois, quatre,
```

Votre fonction `dfs` est récursive, en fait elle utilise la pile d'exécution de c++ pour "stocker" les sommets sur lesquels elle va revenir ensuite. Mais alors ! pour faire une version itérative de cette fonction, il suffit de mettre les sommets dans une pile ! et bien oui parfaitement. Déjà je vous montre la pile en c++. Vous allez voir c'est super facile (simplement `pop` ne renvoie pas l'élément, c'est `top` qui le montre).

```c++
// in file main.cpp
#include <iostream>
#include <stack>

int main (int argc, char* argv []) {
  std::stack<std::string> str_stack;
  str_stack.push("un");
  str_stack.push("deux");
  str_stack.push("trois");
  std::cout << "la pile est vide ? "
	    << std::boolalpha << str_stack.empty() << std::endl;

  std::cout << str_stack.top() << std::endl;
  str_stack.pop();
  std::cout << str_stack.top() << std::endl;
  str_stack.pop();
  std::cout << str_stack.top() << std::endl;
  str_stack.pop();
  std::cout  << "la pile est vide ? "
	     << std::boolalpha << str_stack.empty() << std::endl;

  return 0;
}
```

```c++
$ g++ main.cpp -o stack
$ ./stack
la pile est vide ? false
trois
deux
un
la pile est vide ? true
```

Et maintenant le `dfs_iteratif` du graphe ! Alors vous commencez par créer une pile pour les sommets. Puis vous considérez les sommets de la table de hash les uns après les autres. Vous prenez un sommet, vous le mettez dans la pile, et tant que la pile n'est pas vide, vous popez le sommet du haut de la pile, vous parcourez ses transitions en pushant les sommets d'arrivée sur la pile etc. 

Et maintenant vous avez la version itérative de votre parcours en profondeur. C'est presque terminé. Si à la place de prendre une pile, vous prenez une file (genre file d'attente des gens civilisés où le *premier arrivé est le premier servi*) et bien ... vous faites un parcours en largeur. C'est à dire en prenant les sommets qui sont à 1 transition du sommet de départ, puis à 2 transitions puis à 3 etc. Regardez les explications là:
https://fr.wikipedia.org/wiki/Algorithme_de_parcours_en_largeur.

Si besoin, demandez nous la correction du `dfs_iteratif` et du `bfs` .

END